In [1]:
from __future__ import print_function
import math
#importing keras modules
from keras.models import Sequential
from keras.layers import Dense, Activation ,Dropout , Flatten , Conv1D, Conv2D ,MaxPooling1D
from keras.layers.recurrent import LSTM
from keras import losses
from keras import optimizers

Using TensorFlow backend.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math
import itertools
from sklearn import preprocessing
import datetime
from sklearn.metrics import mean_squared_error, accuracy_score
from math import sqrt

In [3]:
df = pd.read_csv('csv_data/07_feature_engineering_and_cleaning.csv')

In [5]:
df = df.drop(["date", "location"], axis=1)
data = df.as_matrix()

c:\users\tjaka\anaconda3\envs\epidemic_prediction\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [6]:
result = []
sequence_length = 6
for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])
    
result = np.array(result)
row = round(0.8 * result.shape[0])


In [15]:
result = []
sequence_length = 6
for index in range(len(data) - sequence_length):
    result.append(data[index: index + sequence_length])
    
result = np.array(result)
row = round(0.8 * result.shape[0])

#creating training data
train = result[:int(row), :]

x_train = train[:, :-1]
y_train = train[:, -1][:,-1]
x_test = result[int(row):, :-1]
y_test = result[int(row):, -1][:,-1]

amount_of_features = len(df.columns)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features)) 

print("X_train", x_train.shape)
print("y_train", y_train.shape)
print("X_test", x_test.shape)
print("y_test", y_test.shape)

X_train (92882, 5, 23)
y_train (92882,)
X_test (23221, 5, 23)
y_test (23221,)


In [16]:
def build_model(input):
    model = Sequential()
    model.add(Dense(128,input_shape=(input[1],input[0])))
    model.add(Conv1D(filters = 112, kernel_size= 1,padding='valid', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, padding='valid'))
    model.add(Conv1D(filters = 64,kernel_size = 1,padding='valid', activation='relu'))
    model.add(MaxPooling1D(pool_size=1, padding='valid'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(100, activation="relu", kernel_initializer="uniform"))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation="relu", kernel_initializer="uniform"))
    model.compile(loss='mse',optimizer='adam',metrics=['mae'])
    return model

In [17]:
model = build_model([23,5,1])
#Summary of the Model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5, 128)            3072      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 5, 112)            14448     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 112)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 2, 64)             7232      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 2, 64)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)              

In [37]:
from timeit import default_timer as timer
start = timer()
history = model.fit(x_train,
                    y_train,
                    batch_size=256,
                    validation_split=0.2,
                    epochs=10,)
end = timer()
print(end - start)

Train on 74305 samples, validate on 18577 samples
Epoch 1/10
74305/74305 [==============================] - 3s 45us/step - loss: 4.4639 - mae: 0.8070 - val_loss: 14.7629 - val_mae: 2.2997
Epoch 2/10
74305/74305 [==============================] - 3s 44us/step - loss: 3.7291 - mae: 0.7363 - val_loss: 13.9565 - val_mae: 2.2306
Epoch 3/10
74305/74305 [==============================] - 3s 45us/step - loss: 4.1610 - mae: 0.7083 - val_loss: 9.2522 - val_mae: 1.8089
Epoch 4/10
74305/74305 [==============================] - 3s 47us/step - loss: 3.9681 - mae: 0.7148 - val_loss: 16.7574 - val_mae: 2.3493
Epoch 5/10
74305/74305 [==============================] - 3s 45us/step - loss: 4.0274 - mae: 0.6990 - val_loss: 15.2972 - val_mae: 2.3344
Epoch 6/10
74305/74305 [==============================] - 3s 44us/step - loss: 5.2115 - mae: 0.8414 - val_loss: 16.4745 - val_mae: 2.3612
Epoch 7/10
74305/74305 [==============================] - 3s 45us/step - loss: 3.9856 - mae: 0.7523 - val_loss: 13.8518 - v